In [16]:
import csv
import requests
import os
import sys
import warnings
from datetime import datetime, timedelta
warnings.simplefilter('ignore')

In [4]:
caminho_commons = os.path.join('..', '..', 'ajna_docs', 'commons')
sys.path.insert(0, caminho_commons)
from ajna_commons.utils.sanitiza import sanitizar, mongo_sanitizar
DTE_TOKEN = 'https://jupapi.org.br/api/sepes/Pesagem/token'
DTE_URL = 'https://jupapi.org.br/api/sepes/ConsultaFMA'
with open(os.path.join('..', '..', 'virasana', 'virasana', 'integracao', 'dte.info')) as dte_info:
    linha = dte_info.readline()
DTE_USERNAME = os.environ.get('DTE_USERNAME')
DTE_PASSWORD = os.environ.get('DTE_PASSWORD')
DTE_USERNAME = linha.split(',')[0]
DTE_PASSWORD = linha.split(',')[1]

In [5]:
with open(os.path.join('..', '..', 'virasana', 'virasana', 'integracao', 'recintos.csv'), encoding='utf-8') as csv_in:
    reader = csv.reader(csv_in)
    recintos_list = [row for row in reader]

def get_token_dte(username=DTE_USERNAME, password=DTE_PASSWORD):
    data = {'username': username, 'password': password, 'grant_type': 'password'}
    r = requests.post(DTE_TOKEN, data=data, verify=False)
    token = r.json().get('access_token')
    return token


In [12]:
def get_lista_fma(cod_recinto, start, end):
    token = get_token_dte()
    payload = {'data_inicio': datetime.strftime(start, '%Y-%m-%d'),
           'data_fim': datetime.strftime(end, '%Y-%m-%d'),
           'cod_recinto': cod_recinto}
    headers = {'Authorization': 'Bearer ' + token}
    r = requests.get(DTE_URL, headers=headers, params=payload)
    if r.status_code != 200:
        raise Exception(r.text)
    return r.json()['JUP_WS']['FMA_Eletronica']['Lista_FMA']
                  
start = datetime.strptime('01/02/2020', '%d/%m/%Y')
end = datetime.strptime('20/02/2020', '%d/%m/%Y')
cod_recinto = 22
get_lista_fma(cod_recinto, start, end)

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



[{'Sequencial': 1,
  'DT_DTA': 'DT',
  'Numero_FMA': 8,
  'Ano_FMA': 2020,
  'Data_Emissao': '2020-02-03',
  'CPF_Responsavel_Emissao': '19934152894',
  'Cod_Recinto': 22,
  'Nome_Recinto': 'Brasil Terminal Portuario',
  'Cod_Operador': 846,
  'Nome_Operador': 'Brasil Terminal Portuario - Operador',
  'Data_Entrada_Navio': '2019-11-05 11:31:45',
  'Cod_Navio': 9705005,
  'Nome_Navio': 'MSC CATERINA',
  'Cod_Porto': 14,
  'Nome_Porto': 'ANTUERPIA (AMBERES)',
  'Nacionalidade': 'BELGICA',
  'Consignatario': 'BRASIL TERMINAL PORTUARIO S.A.',
  'Numero_Viagem_DTA': 4178,
  'Ano_Viagem_DTA': 2019,
  'CE_Mercante': '151905242759790',
  'BL': 'MEDURP742568',
  'Lista_Containers': [{'Container': 'CXRU1539435'},
   {'Container': 'TTNU8461373'}],
  'Bagagem': 'NÃO',
  'Mercadoria': 'PROD DE OR ANIMAL, N ESP',
  'Embalagem': 'CONTEINER OU COFRE D',
  'Quantidade': 2,
  'Peso': '48240',
  'Unidade_Peso': 'Kg',
  'Marca_Contramarca': '',
  'Obs': ' AINDA NAO HA CONSIGNATARIO DEFINIDO NO SISCOMEX'},

In [7]:
sys.path.append('..')
sys.path.append('../../virasana')

from bhadrasana.models.ovr import Recinto
from bhadrasana.main import session


2020-02-20 15:32:32,335 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ..\..\ajna_docs\commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ..\..\ajna_docs\commons\ajna_commons\flask\access.log


In [8]:
for linha in recintos_list[1:]:
    recinto = Recinto()
    recinto.id = int(linha[0])
    recinto.nome = linha[2]
    try:
        session.add(recinto)
        session.commit()
        print(linha[0], linha[2])
    except:
        session.rollback()
        
    


In [15]:
from bhadrasana.models.ovr import OVR
from datetime import datetime

start = datetime.strptime('01/02/2020', '%d/%m/%Y')
end = datetime.strptime('20/02/2020', '%d/%m/%Y')
for lista in recintos_list[1:]:
    cod_recinto = int(lista[0])
    lista_fma = get_lista_fma(cod_recinto, start, end)
    if lista_fma:
        for fma in lista_fma:
            ovr = OVR()
            ovr.numero = fma['Numero_FMA']
            ovr.ano = fma['Ano_FMA']
            ovr.datahora = datetime.strptime(fma['Data_Emissao'], '%Y-%m-%d')
            ovr.recinto_id = int(fma['Cod_Recinto'])
            ovr.numeroCEmercante = fma['CE_Mercante']
            ovr.tipooperacao = 0
            ovr.fase = 0
            ovr.tipoevento_id = 1
            try:
                session.add(ovr)
                session.commit()
            except Exception as err:
                print(err)
                session.rollback()
     
                                     
# 'CPF_Responsavel_Emissao': '19934152894',
#    'Cod_Operador': 846,
#     'Data_Entrada_Navio': '2019-11-05 11:31:45',
#     'Cod_Navio': 9705005,
#     'Nome_Navio': 'MSC CATERINA',
#     'Cod_Porto': 14,
#     'Nome_Porto': 'ANTUERPIA (AMBERES)',
#     'Nacionalidade': 'BELGICA',
#     'Consignatario': 'BRASIL TERMINAL PORTUARIO S.A.',



C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpoo

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpoo

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

C:\Users\25052288840\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpoo